In [1]:
! pip install -r requirements.txt

In [ ]:
import os
import whisper
from pyannote.audio import Pipeline
from pathlib import Path
import torch
import torchaudio
from openai import OpenAI
from pathlib import Path

# device = "mps" if torch.backends.mps.is_available() else "cpu"
# print("Using device:", device)

#i have used relative paths here 
data_dir = Path("./data_samples")
data_dir.mkdir(exist_ok=True, parents=True)

audio_file = data_dir / "RES0029.mp3"
output_file = data_dir / "transcription3.txt"

print(f"Audio file: {audio_file}")
print(f"Output file: {output_file}")

whisper_model = whisper.load_model("base")

HF_TOKEN = os.getenv("HF_TOKEN")
diarization_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", use_auth_token=HF_TOKEN)


Audio file: data_samples/RES0029.mp3
Output file: data_samples/transcription3.txt


/Users/anushkapawar/Documents/3MORE Work/Medical_call_analysis/.venv/lib/python3.13/site-packages/speechbrain/utils/torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  available_backends = torchaudio.list_audio_backends()
/Users/anushkapawar/Documents/3MORE Work/Medical_call_analysis/.venv/lib/python3.13/site-packages/pyannote/audio/core/io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video ar

In [4]:
diarization = diarization_pipeline(str(audio_file))
segments = []
for turn, _, speaker in diarization.itertracks(yield_label=True):
    segments.append({
        "start": turn.start,
        "end": turn.end,
        "speaker": speaker
    })

print(f"Found {len(segments)} speaker segments.")


/Users/anushkapawar/Documents/3MORE Work/Medical_call_analysis/.venv/lib/python3.13/site-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/Users/anushkapawar/Documents/3MORE Work/Medical_call_analysis/.venv/lib/python3.13/site-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1839.)
  std = sequence

Found 150 speaker segments.


In [5]:
def transcribe_segment(audio_path, start_time, end_time, model):
    waveform, sample_rate = torchaudio.load(audio_path)
    # Trim the audio to the segment
    start_sample = int(start_time * sample_rate)
    end_sample = int(end_time * sample_rate)
    segment_waveform = waveform[:, start_sample:end_sample]

    # Save temporary segment file
    temp_path = "temp_segment.wav"
    torchaudio.save(temp_path, segment_waveform, sample_rate)

    # Transcribe with Whisper
    result = model.transcribe(temp_path)
    return result["text"]

In [6]:
speaker_map = {
    "SPEAKER_00": "Doctor",
    "SPEAKER_01": "Patient"
}
speaker_transcripts = []
for seg in segments:
    text = transcribe_segment(str(audio_file), seg["start"], seg["end"], whisper_model)
    speaker_name = speaker_map.get(seg["speaker"], seg["speaker"])
    speaker_transcripts.append(f'{seg["speaker"]}: {text}')

with open(output_file, "w", encoding="utf-8") as f:
    for line in speaker_transcripts:
        f.write(line + "\n")

print(f"Transcription completed and saved to {output_file}")

/Users/anushkapawar/Documents/3MORE Work/Medical_call_analysis/.venv/lib/python3.13/site-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/Users/anushkapawar/Documents/3MORE Work/Medical_call_analysis/.venv/lib/python3.13/site-packages/torchaudio/_backend/utils.py:337: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.save_with_torchcodec` under the hood. Some parameters like format, encoding, bits_per_sample, buffer_size, and ``backend`` will be ignored. We recommend that you port your code to rely directly on To

Transcription completed and saved to data_samples/transcription3.txt


In [7]:


# Use Hugging Face Router
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=os.environ["HF_TOKEN"],   # Set your HF token here
)

# Example transcript (after Whisper output)
transcript = "\n".join(speaker_transcripts)

# Send to Llama-3.1 for summarization
completion = client.chat.completions.create(
    model="meta-llama/Llama-3.1-8B-Instruct:fireworks-ai",
    messages=[
        {"role": "system", "content": "You are a helpful assistant that summarizes medical conversations."},
        {"role": "user", "content": f"Summarize the following doctor-patient conversation:\n\n{transcript}"}
    ],
)

summary = completion.choices[0].message.content
print(summary)



Here is a summary of the doctor-patient conversation:

The patient, a 28-year-old male, presents to the emergency department with chest pain that started 2 hours ago after falling off his bicycle. He describes the pain as a sharp, stabbing sensation on the left side of his chest, which worsens with deep breathing. He also reports difficulty breathing, feeling like he can't catch his breath, and a racing heart.

The patient has no prior medical conditions, but has a history of allergies and takes medications for them. He is a current smoker, smoking 2-3 packs per day, and occasionally drinks beer on weekends.

The patient's physical examination is notable for a few scrapes on his hands, but no other injuries. He has no fever, chills, nausea, or vomiting.

The doctor asks about the patient's medical history, including any previous hospitalizations or surgeries, but the patient reports none. He also asks about the patient's family medical history, and the patient mentions that his father 